In [ ]:
pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Final_DataFrame_Train.csv')
print(df.head())

In [ ]:
train_sents = df['clean_sent'].to_list()
train_labels = df['tags'].to_list()

print(len(train_sents))
print(len(train_labels))

train_sents = [train_sents[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(train_sents))]
train_labels = [train_labels[i].replace("'", "").strip(
    '][').split(', ') for i in range(len(train_labels))]


print(len(train_sents[0]))
print(len(train_labels[0]))


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-small-uncased")


In [ ]:
# def get_dict_map(labels):
#     """
#     converts word to index

#     [Need changes !!!]
#     """
#     tok2idx = {}
#     idx2tok = {}

#     vocab = []
#     for sent in labels:
#       vocab.extend( list(set(sent)) )
#     vocab = set(vocab)


#     idx2tok = {idx:tok for idx , tok in enumerate(vocab)}
#     tok2idx = {tok:idx for idx , tok in enumerate(vocab)}

#     return tok2idx, idx2tok

# tag2idx, idx2tag = get_dict_map(train_labels)


In [ ]:
def tokenize_and_align_labels(train_sent, train_labels):
    tokenized_input = tokenizer(train_sent, padding='max_length', max_length=512,
                                truncation=True, return_tensors="pt", is_split_into_words=True)

    word_ids = tokenized_input.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(0)
        elif word_idx != previous_word_idx:
            label_ids.append(tag2idx[train_labels[word_idx]])
        else:
            label_ids.append(0)
        previous_word_idx = word_idx

    tokenized_input["labels"] = label_ids
    return tokenized_input

In [ ]:
tokenized_input_and_labels = []
for train_sent, train_label in zip(train_sents, train_labels):
  # print(train_sent)
  # print(train_label)

  tokenized_input_and_labels.append(
      tokenize_and_align_labels(train_sent, train_label))


In [ ]:
print(tokenized_input_and_labels[0]['input_ids'].shape)
print(len(tokenized_input_and_labels[0]['labels']))


In [ ]:
labels = []
for i in range(len(tokenized_input_and_labels)):
  labels.append(tokenized_input_and_labels[i]['labels'])
  del tokenized_input_and_labels[i]['labels']

print(len(labels[0]))

In [ ]:
import torch

hidden_states = []
for input in tokenized_input_and_labels:
  with torch.no_grad():
    outputs = model(**input)
    last_hidden_states = outputs.last_hidden_state
    hidden_states.append(last_hidden_states)

In [ ]:
print(len(hidden_states))
print(hidden_states[0].shape)

In [ ]:
# Saving features and corresponding labels

import pandas as pd

df = pd.DataFrame({"features": hidden_states, "labels": labels})
df.to_pickle("train_features_legalBert.pkl")

In [ ]:
import pandas as pd

print(pd.__version__)

### Load Features and Run Model

In [ ]:
import pandas as pd

df = pd.read_pickle('./bertFeatureExtraction1.pkl')

features = df[['features']].to_numpy()
all_labels = df[['labels']].to_numpy()
all_features = []
for i in range(len(df)):
  all_features.append(features[i][0])

print(len(all_features))
print(all_features[0].shape)
print(all_labels)